## TreResNet on CIFAR
* Use ResNet50, Densenet 121
* best 4 out of 8 networks 
* Compare accuracy for 1 to 4 streams
* Use standard autgmentation, no test time

In [5]:
import os
import sys
import random
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import display, HTML

import scipy.io

%reload_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler

import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50, densenet121
from torchvision import datasets, models, transforms


%reload_ext autoreload
%autoreload 2
%matplotlib inline

PATH = Path('/media/rene/data')

# Add the src directory for functions
src_dir = Path.cwd().parent.parent / 'src'
print(src_dir)
sys.path.append(str(src_dir))

# import my functions:
from utils import make_cfiar10
from functions import*
from models import*

print(torch.cuda.device_count())
print(torch.cuda.is_available())
print(torch.cuda.current_device())

/media/rene/code/idc/src
2
True
0


## Not for this project

In [2]:
# ! wget https://pjreddie.com/media/files/cifar.tgz -P /media/rene/data
# ! tar xzf /media/rene/data/cifar.tgz -C /media/rene/data
# ! mv /media/rene/data/cifar /media/rene/data/cifar_raw

# ! mkdir /media/rene/data/cifar

--2018-07-12 14:23:50--  https://pjreddie.com/media/files/cifar.tgz
Resolving pjreddie.com (pjreddie.com)... 128.208.3.39
Connecting to pjreddie.com (pjreddie.com)|128.208.3.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168584360 (161M) [application/octet-stream]
Saving to: ‘/media/rene/data/cifar.tgz’

cifar.tgz           100%[===================>] 160.77M  47.4MB/s    in 3.4s    

2018-07-12 14:23:54 (47.4 MB/s) - ‘/media/rene/data/cifar.tgz’ saved [168584360/168584360]



In [3]:
# PATH_IN = Path(PATH / 'cifar_raw')
# PATH_OUT = PATH / 'cifar'
# make_cfiar10(PATH_IN, PATH_OUT)

#### Train 8 Individial ResNet 50

## Find 4 best models

In [2]:
save_path = '/media/rene/data/cifar-10-batches-py/models'
PATH = '/media/rene/data'
num_workers = 4
batch_size=64
dataloaders, dataset_sizes = make_batch_gen_cifar(str(PATH), batch_size, num_workers,
                                                   valid_name='valid')

criterion = nn.CrossEntropyLoss()
results = {}

for i in range(8):
    model_name = 'ResNet50_'+str(i)
    model = ResNet50()
    device="cuda:1"
    model = model.to(device)
    
    # load the saved weights
    model_path = os.path.join(save_path, model_name)
    print(model_path)
    model.load_state_dict(torch.load(model_path))
    print('Testing model : ', i)
    valid_loss, valid_acc = eval_model(model, dataloaders['valid'], dataset_sizes['valid'], criterion, device=device)
    results[model_name] = [valid_acc]
    
f = open(os.path.join(save_path, "resnet50_all8.pkl"),"wb")
pickle.dump(results,f)
f.close()

Files already downloaded and verified
Files already downloaded and verified
/media/rene/data/cifar-10-batches-py/models/ResNet50_0
Testing model :  0
Loss: 0.0043 Acc: 0.9405
/media/rene/data/cifar-10-batches-py/models/ResNet50_1
Testing model :  1
Loss: 0.0044 Acc: 0.9376
/media/rene/data/cifar-10-batches-py/models/ResNet50_2
Testing model :  2
Loss: 0.0043 Acc: 0.9380
/media/rene/data/cifar-10-batches-py/models/ResNet50_3
Testing model :  3
Loss: 0.0047 Acc: 0.9360
/media/rene/data/cifar-10-batches-py/models/ResNet50_4
Testing model :  4
Loss: 0.0042 Acc: 0.9375
/media/rene/data/cifar-10-batches-py/models/ResNet50_5
Testing model :  5
Loss: 0.0042 Acc: 0.9412
/media/rene/data/cifar-10-batches-py/models/ResNet50_6
Testing model :  6
Loss: 0.0042 Acc: 0.9392
/media/rene/data/cifar-10-batches-py/models/ResNet50_7
Testing model :  7
Loss: 0.0041 Acc: 0.9399


In [3]:
import operator

# save_path = '/home/rene/data/cifar-10-batches-py/models'

# with open(os.path.join(save_path, "resnet50_all8.pkl"), 'rb') as f:
#     results = pickle.load(f)

results2 = {key:value[0] for (key,value) in results.items()}
sorted_x = sorted(results2.items(), key=operator.itemgetter(1), reverse=True)
print(sorted_x)

[('ResNet50_5', 0.9412), ('ResNet50_0', 0.9405), ('ResNet50_7', 0.9399), ('ResNet50_6', 0.9392), ('ResNet50_2', 0.938), ('ResNet50_1', 0.9376), ('ResNet50_4', 0.9375), ('ResNet50_3', 0.936)]


## Fusion

In [5]:
model_name_list = ['ResNet50_5', 'ResNet50_0', 'ResNet50_7', 'ResNet50_6']
num_workers = 6
device="cuda:0"
batch_size = 32
criterion = nn.CrossEntropyLoss()

PATH = Path('/media/rene/data/')
save_path =  Path(PATH / 'cifar-10-batches-py' / 'models')

dataloaders, dataset_sizes = make_batch_gen_cifar(str(PATH), batch_size, num_workers,
                                                    valid_name='valid')

pretrained_model_list = []
for i, model_name in enumerate(model_name_list):
    model = ResNet50()
    model = model.to(device)
    model.load_state_dict(torch.load(os.path.join(save_path, model_name)))
    pretrained_model_list.append(model)

model = Fusion2(pretrained_model_list, num_input=40, num_output=10)
model.load_state_dict(torch.load(os.path.join(save_path, 'Fusion2_2s1_r2')))
model = model.to(device)

valid_loss, valid_acc = eval_model(model, dataloaders['valid'], dataset_sizes['valid'], criterion)
print('Fusion2_2s1: ', valid_loss, valid_acc)

model = Fusion2(pretrained_model_list, num_input=40, num_output=10)
model.load_state_dict(torch.load(os.path.join(save_path, 'Fusion2_2s2_r2')))
model = model.to(device)

valid_loss, valid_acc = eval_model(model, dataloaders['valid'], dataset_sizes['valid'], criterion)
print('Fusion2_2s2: ', valid_loss, valid_acc)

Files already downloaded and verified
Files already downloaded and verified
Loss: 0.0080 Acc: 0.9496
Fusion2_2s1:  0.00802382983416319 0.9496
Loss: 0.0062 Acc: 0.9541
Fusion2_2s2:  0.006155912631005049 0.9541


## Fusion with more Nodes

In [7]:
model_name_list = ['ResNet50_5', 'ResNet50_0', 'ResNet50_7', 'ResNet50_6']
num_workers = 6
device="cuda:0"
batch_size = 32
criterion = nn.CrossEntropyLoss()

PATH = Path('/media/rene/data/')
save_path =  Path(PATH / 'cifar-10-batches-py' / 'models')

dataloaders, dataset_sizes = make_batch_gen_cifar(str(PATH), batch_size, num_workers,
                                                    valid_name='valid')

pretrained_model_list = []
for i, model_name in enumerate(model_name_list):
    model = ResNet50()
    model = model.to(device)
    model.load_state_dict(torch.load(os.path.join(save_path, model_name)))
    
    # remove last layers
    res50_conv = ResNet50Bottom(model)
    pretrained_model_list.append(res50_conv)

model = Fusion2More(pretrained_model_list, num_input=40, num_output=10)
model.load_state_dict(torch.load(os.path.join(save_path, 'Fusion2_2s1_more')))
model = model.to(device)

valid_loss, valid_acc = eval_model(model, dataloaders['valid'], dataset_sizes['valid'], criterion)
print('Fusion2_2s1: ', valid_loss, valid_acc)

model = Fusion2More(pretrained_model_list, num_input=40, num_output=10)
model.load_state_dict(torch.load(os.path.join(save_path, 'Fusion2_2s2_more')))
model = model.to(device)

valid_loss, valid_acc = eval_model(model, dataloaders['valid'], dataset_sizes['valid'], criterion)
print('Fusion2_2s2: ', valid_loss, valid_acc)

Files already downloaded and verified
Files already downloaded and verified
Loss: 0.0079 Acc: 0.9464
Fusion2_2s1:  0.00793638818487525 0.9464
Loss: 0.0065 Acc: 0.9543
Fusion2_2s2:  0.006539191658049822 0.9543


## Dense Net 121

In [ ]:
PATH = Path('/home/rene/data/cifar/sample')
num_workers = 6
batch_size=32
epochs = 15

models = {}
for i in range(8):
    models['densenet121_'+str(i)] = densenet121
    
dataloaders, dataset_sizes = make_batch_gen(str(PATH), batch_size, num_workers, valid_name='valid', size=224)

for model_name, model_arch in models.items():
    model = model_arch(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, 10)
    model = model.cuda()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.003)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

    best_acc, model = train_model(model, criterion, optimizer, exp_lr_scheduler,
                                     epochs, dataloaders, dataset_sizes)
    torch.save(model.state_dict(), str(PATH /'models' / model_name))